In [151]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import optparse
import requests
from collections import defaultdict

In [114]:
def parse_productcenter(driver, page_num):
    print("Started parsing")
    overall_data = {}
    driver.get(f'https://productcenter.ru/producers/r-moskovskaia-obl-191/c-moskva-3109/page-{page_num}')

    try:
        myElem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ci_main')))
    except TimfirefoxutException:
        print("Loading took too much time!")
    
    elems = driver.find_elements(By.CLASS_NAME, 'ci_main')
    links = []
    for e in elems:
        link = e.find_element(By.CLASS_NAME, 'link')
        link = link.get_attribute('href')
        links.append(link)
    
    
    for link in links:
        comp_data = defaultdict(lambda: '')
        comp_data['link'] = link
        driver.get(link)
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'b')))
        except TimeoutException:
            print("Loading took too much time!")
            continue

        elem = driver.find_element(By.XPATH, '//b[text()="Контакты"]')
        elem.click()
        try:
            e2 = driver.find_element(By.XPATH, '//span[@itemprop="telephone"]')
            comp_data["phone"] = e2.text
        except:
            pass
        
        try:
            e2 = driver.find_element(By.XPATH, '//span[@itemprop="email"]')
            comp_data["email"] = e2.text
        except:
            pass
            
        e = driver.find_element(By.CLASS_NAME, 'company_data')
        table = e.find_elements(By.TAG_NAME, 'tr')
        for tr in table:
            e = tr.find_elements(By.TAG_NAME, 'td')
            key, value = e[0].text, e[1]
            if key in mappings:
                comp_data[key] = mappings[key](value)
        if "Наименование" and "ИНН" in comp_data:
            overall_data[comp_data["Наименование"]] = comp_data
        print(f'Producer has been parsed')
    return overall_data



In [115]:
mappings = {
    "Наименование": lambda e: e.text,
    "Сотрудники": lambda e: int(e.text),
    "ИНН": lambda e: e.text,
}

In [ ]:

parser = optparse.OptionParser()

parser.add_option('-n', '--page-number',
    action="store", dest="page",
    help="page number to parse")

parser.add_option('-l', '--api-link',
    action="store", dest="link",
    help="link to send results")

options, args = parser.parse_args()

print(options)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from time import sleep

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
while True:
    try:
        driver = webdriver.Remote("http://selenium:4444/wd/hub", options=chrome_options)
        break
    except:
        print("Not initialized")
        sleep(5)


res = parse_productcenter(driver, options.page)
driver.close()

In [152]:
def send_user_data(ud):
    requests.post('http://nginx/api/v1/user',
                 json={
                     "url": ud['link'],
                     "email": ud['email'],
                     "phone": ud['phone'],
                     "inn": ud['ИНН'],
                     "login": ud['ИНН'],
                     "description": ud["Наименование"],
                     "role": "ENTERPRISE",
                     "status": "PENDING",
                      }
                 )


In [ ]:
for _, v in res.items():
    send_user_data(v)